In [47]:
!pip install kumaraswamy

In [48]:
from kumaraswamy import kumaraswamy

## Experiments with Kumaraswamy Function

In [49]:
d1 = kumaraswamy(a=0.1, b=0.7)

In [50]:
d1.pdf(0.9)

array(0.30210796)

In [51]:
d1.cdf(0.21)

array(0.7418337)

In [52]:
d1.var

0.07907948744725794

In [53]:
d1.mean

0.17121255852442666

### CCTE

In [54]:
from scipy.special import hyp2f1
from scipy.special import betainc
#The incomplte Beta function


In [55]:
def num_1(alpha,t,a,b,theta):
    v1=-(1/a)
    v2=b
    v3=1+b
    v4=(1-alpha)**(1/b)
##This is the first term in the numerator
    p1=(t-1)*(theta*t-1)
    p2=(1-alpha)*(hyp2f1(v1,v2,v3,v4))
    return p1*p2 #The product of 2  terms is the first term
    

In [56]:
def num_2(alpha,t,a,b,theta):
    #This gives the negative of the second term
    a3=1+(1/a)
    p=alpha**b
    b3=2*b
    p3=2*(theta)*t*(t-1)*b
    p4=[betainc(a3,b,1)+betainc(a3,b3,1)-betainc(a3,b,p)-betainc(a3,b3,p)]
    prod=p4[0]*p3
    return prod


In [57]:
#This defines the FGM Copula for kumaraswamy distribution
# Some checks required
def FGM_copula(alpha,t,a,b,theta,a1,b1):
    d1 = kumaraswamy(a, b)
    d2 = kumaraswamy(a1, b1)
    s1 = d1.cdf(alpha)
    s2 = d2.cdf(t)
    copula=(s1)*(s2)+theta*(s1)*(s2)*(1-s1)*(1-s2)
    return copula
    
    
    
def deno(alpha,t,a,b,theta,a1,b1):
    t1=1-alpha-t
    t2= FGM_copula(alpha,t,a,b,theta,a1,b1)
    return t1+t2
    

In [58]:
FGM_copula(0.3,0.5,0.7,0.8,2,0.6,0.8)

0.32232193347566945

In [59]:
#Here a,b, alpha are for first distribution and a1,b1,t are for second distribution
def ccte_fun(alpha,a,b,t,a1,b1,theta):
    num=num_1(alpha,t,a,b,theta)+num_2(alpha,t,a,b,theta)
    den=deno(alpha,t,a,b,theta,a1,b1)
    return num/den
    

In [60]:
ccte_fun(0.2,0.5,0.8,0.9,0.8,0.9,0.4)

-0.3503350944639705

In [61]:

num_2(0.2,0.7,0.8,0.9,4)

-2.838277127208187

In [62]:
import numpy as np
def matrix(n):
    a=(n,n)
    return np.zeros(a)

In [63]:
c=matrix(3)

In [136]:
c

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35033509, 25.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [64]:
c[1,2]=25

In [65]:
c

array([[ 0.,  0.,  0.],
       [ 0.,  0., 25.],
       [ 0.,  0.,  0.]])

In [69]:
def update_matrix(mat,i,j,alpha,a,b,t,a1,b1,theta):
    #input here whcih element we want to update(exact index)
    mat[i-1,j-1]=ccte_fun(alpha,a,b,t,a1,b1,theta)
    

In [70]:
update_matrix(c,2,2,0.2,0.5,0.8,0.9,0.8,0.9,0.4)

In [71]:
c

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35033509, 25.        ],
       [ 0.        ,  0.        ,  0.        ]])

### The OOP part

In [75]:
class ccte:
    def __init__(self, alpha,a,b,t,a1,b1):
        ## here the a,b represent the first distribution and a1 and b1 correspond to second distribution
        ## alpha is bound for first distribution and t is bound for second distribution
        self.alpha = alpha
        self.a = a
        self.b = b
        self.t = t
        self.a1 = a1
        self.b1 = b1
        
        
    def ccte_calc(self,theta):
        alpha=self.alpha
        a=self.a
        b=self.b
        t=self.t
        a1=self.a1
        b1=self.b1
        
        return ccte_fun(alpha,a,b,t,a1,b1,theta)

In [76]:
obj=ccte(0.2,0.5,0.8,0.9,0.8,0.3)

In [77]:
obj.ccte_calc(0.2)

-0.21156365084895895

In [153]:
class dispersion_matrix:
    
    def __init__(self,n):
        self.n=n
        #self.matrix=(n,n)
        #np.zeros(self.matrix)
        
    def matrix_def(self):
        self.matrix=(self.n,self.n)
        self.p=np.zeros(self.matrix)
        return self.p
    
    def update_mat(self,i,j,alpha,a,b,t,a1,b1,theta):
        self.p[i-1,j-1]=ccte_fun(alpha,a,b,t,a1,b1,theta)
        return self.p


In [154]:
obj2=dispersion_matrix(3)

In [155]:
obj2.matrix_def()

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [156]:
obj2.update_mat(2,2,0.2,0.5,0.8,0.9,0.8,0.9,0.4)

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35033509,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])